In [23]:
# imports
from keras.datasets import mnist
from matplotlib import pyplot as plt
import random
import numpy as np

# **3. Load the mnist dataset**
***

In [24]:
# define random seed
random.seed(42)
# define train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# **4. Present simple statistics of this dataset:**
***

In [25]:
def show_statistics(dataset):
    """
    Shows some statistics about the dataset
    :param dataset: 
    :return: 
    """
    # load the dataset
    (input_train, labels_train),(input_test,labels_test) = dataset.load_data()
     
     # combine training and test
    images = np.concatenate((input_train, input_test), axis=0)
    labels = np.concatenate((labels_train, labels_test), axis=0)
    
    # number of images
    num_images = len(images)
    
    # show the distribution of the images
    class_distribution = np.bincount(labels)
    plt.title(f'Distribution of images in the {dataset.__name__} dataset')
    plt.xlabel('class label', fontsize=14)

    plt.ylabel('Count', fontsize=14)
    plt.hist(class_distribution)
    plt.show()
    
    # average number of white pixels(255) and their std per class
    average_white_pixels = []
    std_white_pixels = []
    common_non_white_pixels = []
    
    for digit in range(10): # 10 digits 0-9:
        digit_images = images[labels == digit]
        white_pixels_count = np.sum(digit_images == 255, axis=(1, 2)) # columns of pixels, with consideration to RGB matrix
        average_white_pixels.append(np.mean(white_pixels_count))
        std_white_pixels.append(np.std(white_pixels_count))
        
        #calculate common non-white pixels
        common_non_white = np.all(digit_images!= 255, axis =0).sum()    # if any pixel is not 255, he is not white
        common_non_white_pixels.append(np.sum(digit_images == 255, axis=(1, 2)))
        
    
    
    

IndentationError: unexpected indent (3527360215.py, line 11)